# Generate noisy signals

The default langmuir samples are not noisy and, thus, do not properly reflect real world data.  This notebook will add noise to the signals to help simulate real world data.

In [8]:
import astropy.units as u
import numpy as np
import os

from plasmapy.utils.decorators import validate_quantities

In [9]:
# noise generator
@validate_quantities
def add_noise(signal, snr_db: u.dB = 10 * u.dB):
    sig_pwr = signal ** 2
    sig_pwr_ave = np.mean(sig_pwr)
    sig_pwr_ave_db = 10. * np.log10(sig_pwr_ave)
    
    noise_pwr_ave_db = sig_pwr_ave_db - snr_db.value
    noise_pwr_ave = 10.0 ** (noise_pwr_ave_db / 10.0)
    
    noise = np.random.normal(0.0, np.sqrt(noise_pwr_ave), signal.size)
    
    return signal + noise

In [10]:
# read in example data
# this can be found at 
#   https://github.com/PlasmaPy/PlasmaPy/blob/master/docs/notebooks/langmuir_samples/Beckers2017.npy
#
filename = "Beckers2017.npy"
samples_path = os.path.join(os.path.abspath("./"))
path = os.path.join(samples_path, filename)
voltage, current = np.load(path)

# add some artificial noise to simulate reliztic digitized signals
voltage = add_noise(voltage, 38 * u.dB)
current = add_noise(current, 28 * u.dB)

In [11]:
save = False
if save:
    save_path = os.path.join(samples_path, filename + "_noisy.npy")
    with open(save_path, "wb") as fp:
        np.save(fp, [voltage, current])